<h1>
    <center>
        TMA4320 Prosjekt 2

Formålet med prosjektet er å gi en introduksjon til maskinlæring, nærmere bestemt dyplæring og dype nevrale nettverk, ved egen implementering av algoritmene. 
Ideen er å trene dataprogrammet til å gjenkjenne bildeelementer i to ulike testproblemer. I det første testproblemet er problemstillingen forenklet til gjenkjenning av fargede datapunkt, mens i det andre testproblemet skal programmet kunne skille mellom to ulike håndskrevne sifre. 
Denne rapporten skal først presentere den grunneleggende algoritmen som benyttes i treningsfasen og hovedsaklig er lik for begge problemstillingene, før hver problemstilling blir framstilt i detalj.
Valg av startparametre skal begrunnes og parametrene som ble funnet i treningsfasen skal presenteres. Deretter skal modellen med disse fikserte parametrene kjøres med nye data og resultatene bli lagt fram og diskutert.

In [30]:
import numpy as np
import math
import random 

In [46]:
def testOppg1():
    features, labels = read_spiral_from_file()
    W_k, b_k = makeRandomParam()
    Z_vec = labels*1
    Y_k = [features]
    for i in range(W_k.shape[1]):
        Y_next = findNextYMat(Y_k[-1], W_k[i], b_k[i])
        Y_k.append(Y_next)
    #Nå inneholder Y_k ALLE k, dvs at Y_end er med i Y_k, og må derfor skilles ut før Y_k settes inn i 
    #Gradientfunksjonen
    print(Z_vec)

testOppg1()

[1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1
 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0
 0 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 1
 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 0 1 0
 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 0
 0 0 1 0 0 1 1 1 0 1 1 1 1 1 0]


In [42]:
#Lager w_k og b_k

def makeRandomParam(d=2, K=3):
    w_k=np.random.randn(K,d,d)
    b_k=np.random.randn(K,d)
    return w_k,  b_k

In [32]:
h = 1
#Definerer konstant for antall lag:
K = 2

#Finner Y_k+1
#Trenger først sigma, aktiveringsfunksjonen
def sigma(x):
    return(np.tanh(x))
#     Dersom vi heller vil bruke max(), nå må x kun være vektor, denne må utvides:
#     Denne er brukt til å teste funksjonene
#     if isinstance(x, (np.ndarray, list)):
#         returliste = []
#         for element in x:
#             returliste.append(max(element, 0))
#         return(np.array(returliste))
#     else:
#         return(max(0, x))

#Sigma derivert analytisk
def sigma_diff(x):
    return(1-np.tanh(x)**2)

def eta(x):
    return(np.exp(x)/(np.exp(x)+1))

#Analytisk løsning av derivasjon av eta
def eta_diff(x):
    return(np.exp(x)/(np.exp(x) + 1)**2)
    
def checkArray(array, string):
    if isinstance(array, list):
        return(np.array(array))
    elif not isinstance(array, np.ndarray):
        raise ValueError(string, "was not an np.ndarray.")
        
#Hjelpefunskjon for å regne ut vektorproduktet V_1@V_2^T
#Denne vil altså returnere en matrise, ikke en skalar størrelse
def makeMatrixFromVec(a, b):
    return_matrix = np.zeros((len(a), len(b)))
    for i in range(len(b)):
        return_matrix[:, i] = b[i]*a
    return(return_matrix)
    
def findNextYVec(Y_k, W_k, b_k):
    #Denne funksjonen skal finne Y_k+1 = Y_k + h sigma(W_k * Y_k + b_k)
    #Y er en matrise (dxI), W er en matrise (dxd) og b er en vektor
    #Her er sigma en skalar funksjon som anvendes komponentvis på vektoren.
    #All input skal være np.ndarrays, men b_k vil være den eneste som lager krøll
    checkArray(b_k, 'b_k')
    # Finner produktet W_k * Y_k og legger til b_k
    vec_for_sigma = np.dot(W_k, Y_k) + b_k
    #Anvender sigma elementvis
    sigma_vec = sigma(vec_for_sigma)
    #Avkommenter for å sjekke at sigma gjør som den skal
    #print(vec_for_sigma)
    #print(sigma_vec)
    return(Y_k+h*sigma_vec)

def findNextYMat(Y_k_mat, W_k, b_k):
    #Shape[1] finner antall kolonner
    #Shape[0] finner antall rader, dvs. samme som len()
    numberOfColumns = Y_k_mat.shape[1]
    nextYMat = np.array([np.array([0.0 for i in range(numberOfColumns)]) for e in range(Y_k_mat.shape[0])])
    for column in range(numberOfColumns):
        Y_k = Y_k_mat[:, column]
        nextYMat[:, column] = findNextYVec(Y_k, W_k, b_k)
    return(np.array(nextYMat))

#Legger ved matrisene jeg har brukt for å sjekke at alt fungerer som det skal
A = np.array([[1, 2], [2, 1]])
Y_k_mat = np.array([np.array([-2, 1, 1]), np.array([1, 1, 1])])
print(findNextYMat(Y_k_mat, A, np.array([1, 1])))
#Her er altså b_k = [1, 1]

#Hjelpefunksjon som vi strengt tatt definerer på nytt senere, men det er denne jeg har brukt i testene
def findZvalue(Y_k_mat, w_vec, my):
    #w og my er konstante mellom alle K-lag, men endres for for iterasjon
    #Skal regne ut eta(Y_k^T * w + my * [1 for i in range(len(w))]
    vec_for_eta = np.dot(w_vec, Y_k_mat) + my #np.array([my for i in range(Y_k_mat.shape[1])])
    #Kan også regne ut:
    #Y_T = np.transpose(Y_k_mat)
    #vec_for_eta = np.dot(Y_T, w) + np.array([w for i in range(len(w))])
    #print(vec_for_eta)
    return(eta(vec_for_eta))

def findGradJ(Z_vec, W_K, Y_K, Y_end, w_vec, c_vec, my, b_K):
    #Til kodeoptimalisering - et produkt går igjen: eta_diff(Y_K^T * w_vec + my*Vec1)
    prod_to_re_use = eta_diff(np.dot(w_vec, Y_end)+my) #np.array([my for i in range(len(Z_vec))]))
    print("J_diff_my: ", np.dot(prod_to_re_use, Z_vec-c_vec))
    #Dette kommer egentlig til slutt, men starter der:
    #Z befinner seg inni U
    J_diff_my = 0
    for i in range(len(Z_vec)):
        J_diff_my += (Z_vec[i]-c_vec[i])*eta_diff(np.dot(w_vec, Y_end[:, i])+my)
    print(J_diff_my)
    #J_diff_my sjekket for hånd!
    #dJ/dw  = Y_K * [(Z-c)had*eta_diff(Y_K^T*w + my*1Vec]
    #had* er hadamard-produktet, som bare er å gange sammen alle elementene for seg, 1Vec er en vektor fylt med enere
    diff_my_vec_for_product = np.multiply(Z_vec - c_vec, eta_diff(np.dot(w_vec, Y_end)+my))
    J_diff_w = np.dot(Y_end, diff_my_vec_for_product)
    print("J_diff_my: ", J_diff_w)
    #Definerer P_K = dJ/dY_K
    #P_K er altså en matrise
    #dJ/dY_K = w_vec * [(Z_vec - c_vec)had*eta_diff(Y_K^T * w_vec + my*Vec1)]^T      NB: Husk transponering
    #Rekursivt:
    #P_k-1: P_k + h*W_k-1^T*[sigma_diff(W_k-1*Y_k-1 + b_k-1)had*P_k)
    P_K = makeMatrixFromVec(w_vec, np.multiply(Z_vec - c_vec, prod_to_re_use))
    #Ønsker nå å samle opp for alle de deriverte k < K:
    P_k_liste = [P_K]
    #Ønsker å lage b_k som en matrise, b_k har samme dimensjon som en kolonne av WdotY, men den må være like lang
    for i in range(K-1, -1, -1):
        WdotY = np.dot(W_K[i], Y_K[i])
        b_i_mat = np.array([np.array([b_K[j][i] for e in range(WdotY.shape[1])]) for j in range(WdotY.shape[0])])
        matrix_for_sigma_diff =  WdotY + b_i_mat #Transponer denne!
        matrix_for_product = np.multiply(sigma_diff(matrix_for_sigma_diff), P_k_liste[0])
        P_i = P_k_liste[0] + h * np.dot(np.transpose(W_K[i]), matrix_for_product)
        P_k_liste.insert(0, P_i)
    print("P_k_liste: ", P_k_liste)
    #Har nå funnet alle P_k, og kan begynne å regne ut de deriverte.
    J_diff_W_matrix = []
    #dJ/dW_k = h*[P_k+1 *had sigma_diff(W_k*Y_k + b_k)]*Y_k^T
    J_diff_b_vec = []
    #dJ/db_k = h*[P_k+1 *had sigma_diff(W_k*Y_k + b_k)]*
    for j in range(K):
        W_jY_j = np.dot(W_K[j], Y_K[j])
        #print("W_jY_j: ", W_jY_j)
        b_j_mat = np.array([np.array([b_K[c][j] for e in range(W_jY_j.shape[1])]) for c in range(W_jY_j.shape[0])])
        #print("b_j_mat: ", b_j_mat)
        one_vector = np.array([1.0 for e in range(W_jY_j.shape[1])])
        J_diff_W_matrix.append(h*np.dot(P_k_liste[j+1]*sigma_diff(W_jY_j + b_j_mat), np.transpose(Y_K[j])))
        J_diff_b_vec.append(h*np.dot(P_k_liste[j+1]*eta_diff(W_jY_j + b_j_mat), one_vector))
    #print("J_diff_W_matrix: ", J_diff_W_matrix)
    #print("J_diff_b_vec: ", J_diff_b_vec)
    return(J_diff_W_matrix, J_diff_b_vec, J_diff_w, J_diff_my)

#Notat: Nå har jeg lagt inn gradientfunksjonen, og den kjører i pycharm, men ikke alle returverdier er sjekket for hånd
#Her er testene heg har brukt
Y_0 = np.array([np.array([1.0,0.0,2.0]), np.array([1, 1, 3])])
W_2 = [np.array([np.array([1.0, 2.0]), np.array([0.0, 1.0])]), np.array([np.array([1.0, 1.0]),np.array([1.0, 2.0])])]
b_2 = [np.array([0.0, 1.0]), np.array([1.0, 1.0])]
w = np.array([1.0, 1.0])
my = 1
c_vec = np.array([1, 0, 1])

#Har regnet at Y_1 stemmer for hånd
Y_1 = findNextYMat(Y_0, W_2[0], b_2[0])
Y_2 = findNextYMat(Y_1, W_2[1], b_2[1])
print("Y_1: ", Y_1)
print("Y_2: ", Y_2)

Y_K2 = np.array([Y_0, Y_1])
Y_end = Y_2

Z_vec = findZvalue(Y_end, w, my)
print("Z:",Z_vec)
findGradJ(Z_vec, W_2, Y_K2, Y_end, w, c_vec, my, b_2)


[[-1.23840584  1.9993293   1.9993293 ]
 [ 0.03597242  1.9993293   1.9993293 ]]
Y_1:  [[1.99505475 0.96402758 2.99999977]
 [1.96402758 1.96402758 3.9993293 ]]
Y_2:  [[2.99495622 1.96325312 3.99999955]
 [2.96402564 2.96401233 4.9993293 ]]
Z: [0.99905084 0.99734133 0.99995457]
J_diff_my:  0.0026436538598699254
0.0026436538598699254
J_diff_my:  [0.00518923 0.00783582]
P_k_liste:  [array([[-9.09115273e-07,  2.83586627e-03, -2.06364276e-09],
       [-9.81602990e-07,  3.21022271e-03, -2.06641093e-09]]), array([[-9.00233535e-07,  2.64873122e-03, -2.06364183e-09],
       [-9.00237026e-07,  2.64881187e-03, -2.06364183e-09]]), array([[-9.00052673e-07,  2.64455598e-03, -2.06364090e-09],
       [-9.00052673e-07,  2.64455598e-03, -2.06364090e-09]])]


([array([[-8.88173937e-09,  1.87126164e-04],
         [-6.36080230e-08,  1.87077133e-04]]),
  array([[3.94694896e-06, 8.04154980e-06],
         [7.77425248e-08, 1.58393324e-07]])],
 [array([0.00027806, 0.00027801]), array([5.00537469e-05, 7.26115617e-06])],
 array([0.00518923, 0.00783582]),
 0.0026436538598699254)

In [34]:
#Lager Z
def n(x):
    return (np.exp(x))/(np.exp(x)+1)

def Z(Y_k, w, u, n):
    vec=Y_k.transpose()@w + u*np.ones(((Y_k.transpose()@w).size), dtype=int)
    return n(vec)

#print(Z(Y_k,w,u,n))

In [35]:
#lager J

def J(Z,c):
    return 0.5*np.linalg.norm(Z-c)**2


In [36]:
#lager neste U, dvs. parametrene

def U_jPLussEn(W_k,b_vec,w,my,tau,J_diff_W_matrix, J_diff_b_vec, J_diff_w, J_diff_my):
    W_k = W_k - tau*J_diff_W_matrix
    b_vec = b_vec - tau*J_diff_b_vec
    w = w - tau*J_diff_w
    my = my -tau*J_diff_my
    return W_k, b_vec,w,my

In [37]:
#adam descent

                                                                                                                             
def adam(m,v,j,J_diff_W_matrix, J_diff_b_vec, J_diff_w, J_diff_my,W_k,b_vec,w,my):                                           
    beta1 = 0.9                                                                                                              
    beta2 = 0.999                                                                                                            
    alfa = 0.01                                                                                                              
    epsilon = 1E-8                                                                                                           
                                                                                                                             
    G = [J_diff_W_matrix,J_diff_b_vec,J_diff_w,J_diff_my]                                                                    
    U = [W_k,b_vec,w,my]                                                                                                     
                                                                                                                             
    for i in range(len(G)):                                                                                                  
                                                                                                                             
        m = m + (1-beta1)*G[i]                                                                                               
        v = v + (1-beta2)*(g[i]*g[i])                                                                                        
                                                                                                                             
        mj = m/(1-beta1**j)                                                                                                  
        vj = v/(1-beta2**j)                                                                                                  
                                                                                                                             
        U[i] = U[i] - alfa*(mj/(np.sqrt(vj)+epsilon))                                                                        
                                                                                                                             
    return W_k,b_vec,w,my, m,v                                                                                               

In [45]:
def write_spiral_to_file(filename = 'spiral_text_file.txt'):
    features, labels = get_data_spiral_2d()
    try:
        file = open(filename, 'x')
    except FileExistsError:
        file = open(filename, 'w')
    features_0 = ''
    features_1 = ''
    labels_str = ''
    #Itererer gjennom nesten hele datasettet, fordi siste element skal håndteres litt annerledes
    for n in range(len(features[0]) - 1):
        features_0 += str(features[0][n]) + ', '
        features_1 += str(features[1][n]) + ', '
        labels_str += str(labels[n][0]) + ', '
    features_0 += str(features[0][len(features[0]) - 1]) + '\n'
    features_1 += str(features[1][len(features[0]) - 1]) + '\n'
    labels_str += str(labels[len(features[0]) - 1][0]) + '\n'
    print(features_0, features_1, labels_str)
    file.write(features_0)
    file.write(features_1)
    file.write(labels_str)
    file.close()
    
def read_spiral_from_file(filename = 'spiral_text_file.txt'):
    #Denne funksjonen leser en fil som er lagret på formatet fra write_spiral_to_file
    file = open(filename, 'r')
    #Vet at filen lagres med tre linjer
    features_0 = file.readline()
    features_0.strip()
    features_list_0 = features_0.split(', ')
    features_1 = file.readline()
    features_1.strip()
    features_list_1 = features_1.split(', ')
    labels_str = file.readline()
    labels_list = labels_str.split(', ')
    for i in range(len(labels_list)):
        features_list_0[i] = float(features_list_0[i])
        features_list_1[i] = float(features_list_1[i])
        if labels_list[i] == 'True':
            labels_list[i] = True
        else:
            labels_list[i] = False
    return(np.array([features_list_0, features_list_1]), np.array(labels_list))

#write_spiral_to_file()

#features, labels = read_spiral_from_file()
#print(features, labels)

## Inkluderte funksjoner

In [39]:
def get_data_spiral_2d(n_samples=200):
    '''Create "observations" for training.
    
    Creates n_samples dots on two intertwined spirals.
    The dots are classified based on which spiral they belong to and
    then fuzzed by perturbing the coordinates and permuting the ordering.'''
    m1 = math.ceil(n_samples / 2)
    m2 = n_samples - m1
    
    n_turns = 1.0
    
    phi1 = np.pi
    d1 = make_spiral(m1, phi1, n_turns)

    
    phi2 = (phi1 + np.pi) % (2.0 * np.pi)
    d2 = make_spiral(m2, phi2, n_turns)
    
    features = np.hstack((d1, d2))
    labels = np.ones((n_samples,1), dtype='bool_')
    labels[m1:] = False
    
    features = features + .05 * np.random.randn(*features.shape)
    
    indexes = np.random.permutation(n_samples)
    features = features[:, indexes]
    labels = labels[indexes]


    return features, labels
print(get_data_spiral_2d())

def make_spiral(m, phi, n_turns):
    '''Makes points on a spiral
    
    This is a utility function for get_data_spiral_2d'''
    r = np.linspace(0.1, 1.0, m)
    a = np.linspace(0.1, 2.0 * np.pi * n_turns, m)
    xs = r * np.cos(a + phi)
    ys = r * np.sin(a + phi)
    return np.stack([xs, ys])


(array([[-3.68338558e-01, -6.01057454e-01,  1.35053032e-02,
         5.02366970e-01,  2.99019221e-01,  3.62955612e-01,
        -7.59241826e-02, -6.22563646e-01, -3.59360049e-01,
         6.82060802e-01,  1.07228479e-01, -2.09314577e-02,
         5.01240891e-01,  2.95714371e-01,  5.81011021e-01,
         1.13761239e-01,  5.23556972e-01, -2.99633738e-01,
        -4.53629724e-01, -5.60481106e-01, -3.05472052e-01,
         1.08625636e-01,  3.92765045e-01, -1.60046623e-01,
        -4.77344545e-01,  6.18487529e-02, -6.49789045e-01,
         5.53763421e-01, -6.42053130e-02,  7.49951433e-02,
        -6.02021930e-02,  3.77560732e-02,  1.89881661e-01,
         4.11854554e-01, -5.69303072e-01,  5.51705102e-02,
        -2.75034092e-01,  4.25926179e-01, -5.60320843e-01,
         3.04745973e-01,  9.45851596e-03, -8.49991229e-02,
        -8.92778391e-01,  7.70201233e-01,  2.17106430e-01,
         3.20987568e-02,  4.47969696e-01, -4.23870631e-01,
         9.45487815e-01,  5.35953488e-01, -2.20124454e-